In [1]:
import pandas as pd
import numpy as np

from recommendation import AbstractRecommender

pageview = pd.read_csv("../data/pageview.csv", error_bad_lines=False)
bow_content_df = pd.read_csv("../data/article_bag_of_words.csv")

In [2]:
class Recommender(AbstractRecommender):
    def __init__(self):
        self.dataset = pageview
        
    def observe(self, user_interaction):
        pass
        
    def most_recent_url(self, data, user_id):
        subset = data[data["USER_ID"] == user_id]
        url = subset.sort_values("ROW_NUM", ascending=True).iloc[0]["URL_PATH"]
        return url
    
    def cosine_similar(self, x,y):
        x_meta = x.drop(['url'])
        y_meta = y.drop(['url'])

        return(x_meta@y_meta/((x_meta@x_meta)**0.5*(y_meta@y_meta)**0.5))
    
    def find_most_similar_article(self, url, n):
        article = bow_content_df.loc[bow_content_df['url']==url,:]
        others = bow_content_df.loc[bow_content_df['url']!=url,:]
        similarities = others.apply(lambda x:self.cosine_similar(x,article.iloc[0,:]), axis = 1)
        top = similarities.nlargest(n).index
        return others.loc[top, "url"]

    def recommend(self, user_id, n):
        recent_url = pd.Series(self.most_recent_url(self.dataset, user_id), dtype="str")
        recent_url = recent_url.str.replace("/en", "")
        if recent_url.values == "":
            return self.dataset["URL_PATH"].value_counts().nlargest(n + 1)[1:].index.tolist()
        else:
            return self.find_most_similar_article(recent_url.values[0], n)
        
rec = Recommender()

In [3]:
rec.recommend('5150228a-ad62-4a14-9169-79e7979e1a5e_ae8f9f75-df4e-4bbd-82f9-a73cea7fded9', 5)

['/en/friends-feelings/empowering-your-child/building-on-strengths/types-of-strengths-in-kids',
 '/en/tools/through-your-childs-eyes/player',
 '/en/school-learning/special-services/504-plan/the-difference-between-ieps-and-504-plans',
 '/en/learning-attention-issues/signs-symptoms/developmental-milestones/developmental-milestones-for-typical-2-year-olds',
 '/en/learning-attention-issues/child-learning-disabilities/add-adhd/adhd-and-emotions-what-you-need-to-know']

In [71]:
rec.recommend('5150228a-ad62-4a14-9169-79e7979e1a5e_75a81881-b74a-46f1-b577-011eb936e3af', 3)

/school-learning/your-childs-rights/basics-about-childs-rights/school-vouchers-what-you-need-to-know
/school-learning/your-childs-rights/basics-about-childs-rights/school-vouchers-what-you-need-to-know


1277    /school-learning/choosing-starting-school/find...
1275    /school-learning/choosing-starting-school/find...
1206    /school-learning/special-services/ieps/underst...
Name: url, dtype: object